In [1]:
import os
import cv2
import json
import copy
import numpy as np
import streamlit as st
import matplotlib.pyplot as plt
from inference.infer import PoseDetector, ActionClassifier
from inference.rules import Rules
import pandas as pd

In [2]:
os.chdir('../')

In [90]:
os.listdir('./data_folder/labels/pro_jsons/')

['0.json',
 '1.json',
 '10.json',
 '11.json',
 '12.json',
 '13.json',
 '14.json',
 '15.json',
 '16.json',
 '17.json',
 '18.json',
 '2.json',
 '3.json',
 '4.json',
 '5.json',
 '6.json',
 '7.json',
 '8.json',
 '9.json']

In [4]:
os.getcwd()

'D:\\ML\\ai_golf\\teamproject_golf'

In [101]:
test = {}

In [102]:
test['address'] = address_df.describe().round(2).iloc[1:3, :].to_dict()

In [103]:
test['backswing'] = backswing_df.describe().round(2).iloc[1:3, :].to_dict()

In [104]:
test['top'] = top_df.describe().round(2).iloc[1:3, :].to_dict()

In [105]:
test['impact'] = impact_df.describe().round(2).iloc[1:3, :].to_dict()

In [106]:
test['follow'] = follow_df.describe().round(2).iloc[1:3, :].to_dict()

In [107]:
test

{'address': {'어깨선각도': {'mean': 12.2, 'std': 6.14},
  '양손위치': {'mean': 51.31, 'std': 11.65},
  '스탠스': {'mean': 57.54, 'std': 7.35},
  '상체기울임정도': {'mean': 185.82, 'std': 7.26}},
 'backswing': {'왼쪽어깨회전': {'mean': 50.99, 'std': 5.98},
  '왼팔펴짐각도': {'mean': 152.09, 'std': 8.95}},
 'top': {'하체고정': {'mean': 43.82, 'std': 4.02},
  '오른쪽다리펴짐각도': {'mean': 175.32, 'std': 3.8},
  '스웨이체크': {'mean': 33.43, 'std': 6.56}},
 'impact': {'행잉백': {'mean': 7.57, 'std': 2.6},
  '오른쪽K라인각도': {'mean': 144.65, 'std': 11.33},
  '왼팔펴짐각도': {'mean': 174.42, 'std': 4.63},
  '오른팔펴짐각도': {'mean': 162.68, 'std': 5.92}},
 'follow': {'왼쪽라인': {'mean': 4.06, 'std': 2.69},
  '치킨윙': {'mean': 163.68, 'std': 8.81},
  '체중전진이동': {'mean': 68.0, 'std': 6.23},
  '오른쪽다리펴짐각도': {'mean': 162.51, 'std': 8.55}}}

In [108]:
buf = './data_folder/parameter/label_rule_value.json'
with open(buf, 'w',encoding='utf-8') as f:
    json.dump(test, f, indent=4, ensure_ascii=False)


In [95]:
rules = []
for i in os.listdir('data_folder/labels/pro_jsons/'):
    with open(os.path.join('./data_folder/labels/pro_jsons/', i), 'rb') as r:
        rules.append(json.load(r))

In [109]:
address_df = pd.DataFrame()
for rule in rules[1:]:
    if len(address_df) == 0:
        address_df = pd.DataFrame(data=[list(rule['address'].values())],
                                 columns=list(rule['address'].keys()))
    else:
        new = pd.DataFrame(data=[list(rule['address'].values())],
                                 columns=list(rule['address'].keys()))
        address_df = pd.concat([address_df, new])

address_df.reset_index(inplace=True,drop=True)
address_df.describe()

,어깨선각도,양손위치,스탠스,상체기울임정도
count,9.000000,9.000000,9.000000,9.000000
mean,10.593333,54.694444,56.708889,185.031111
std,3.672383,4.880318,7.284994,7.231050
min,6.780000,48.780000,48.590000,175.610000
25%,7.790000,52.180000,50.290000,180.830000
50%,9.610000,52.790000,56.200000,185.470000
75%,15.210000,56.520000,61.560000,187.540000
max,15.320000,65.640000,70.050000,198.050000


In [110]:
backswing_df = pd.DataFrame()
for rule in rules:
    if len(backswing_df) == 0:
        backswing_df = pd.DataFrame(data=[list(rule['backswing'].values())],
                                 columns=list(rule['backswing'].keys()))
    else:
        new = pd.DataFrame(data=[list(rule['backswing'].values())],
                                 columns=list(rule['backswing'].keys()))
        backswing_df = pd.concat([backswing_df, new])

backswing_df.reset_index(inplace=True,drop=True)
backswing_df.describe()

,왼쪽어깨회전,왼팔펴짐각도
count,10.000000,10.000000
mean,50.987000,152.086000
std,5.977748,8.953288
min,45.630000,142.790000
25%,46.540000,144.647500
50%,48.360000,149.125000
75%,54.585000,159.445000
max,61.050000,166.690000


In [111]:
top_df = pd.DataFrame()
for rule in rules:
    if len(top_df) == 0:
        top_df = pd.DataFrame(data=[list(rule['top'].values())],
                                 columns=list(rule['top'].keys()))
    else:
        new = pd.DataFrame(data=[list(rule['top'].values())],
                                 columns=list(rule['top'].keys()))
        top_df = pd.concat([top_df, new])

top_df.reset_index(inplace=True,drop=True)
top_df.describe()

,하체고정,오른쪽다리펴짐각도,스웨이체크
count,10.000000,10.000000,10.000000
mean,43.825000,175.317000,33.428000
std,4.020653,3.800333,6.560985
min,37.990000,167.150000,24.220000
25%,41.827500,174.350000,30.145000
50%,43.975000,176.060000,31.370000
75%,45.830000,177.577500,39.027500
max,51.570000,179.580000,42.920000


In [112]:
impact_df = pd.DataFrame()
for rule in rules:
    if len(impact_df) == 0:
        impact_df = pd.DataFrame(data=[list(rule['impact'].values())],
                                 columns=list(rule['impact'].keys()))
    else:
        new = pd.DataFrame(data=[list(rule['impact'].values())],
                                 columns=list(rule['impact'].keys()))
        impact_df = pd.concat([impact_df, new])

impact_df.reset_index(inplace=True,drop=True)
impact_df.describe()

,행잉백,오른쪽K라인각도,왼팔펴짐각도,오른팔펴짐각도
count,10.000000,10.000000,10.000000,10.000000
mean,7.572000,144.646000,174.425000,162.683000
std,2.599161,11.328906,4.631398,5.916608
min,4.080000,128.770000,167.480000,155.220000
25%,5.600000,141.797500,170.522500,160.295000
50%,7.285000,143.290000,176.430000,160.720000
75%,9.942500,148.565000,177.720000,164.437500
max,11.500000,165.950000,178.980000,174.270000


In [113]:
follow_df = pd.DataFrame()
for rule in rules:
    if len(follow_df) == 0:
        follow_df = pd.DataFrame(data=[list(rule['follow'].values())],
                                 columns=list(rule['follow'].keys()))
    else:
        new = pd.DataFrame(data=[list(rule['follow'].values())],
                                 columns=list(rule['follow'].keys()))
        follow_df = pd.concat([follow_df, new])

follow_df.reset_index(inplace=True,drop=True)
follow_df.describe()

,왼쪽라인,치킨윙,체중전진이동,오른쪽다리펴짐각도
count,10.000000,10.000000,10.000000,10.000000
mean,4.055000,163.676000,68.000000,162.507000
std,2.686072,8.808622,6.229246,8.545918
min,1.140000,144.380000,54.740000,146.030000
25%,2.002500,159.605000,65.395000,158.240000
50%,3.580000,166.220000,70.990000,163.355000
75%,5.567500,167.560000,71.962500,170.202500
max,8.700000,174.620000,74.690000,171.590000


In [91]:
# ----------------------------------------------------------

rule = Rules()
with open('data_folder/parameter/params.json', 'r') as r:
    params = json.load(r)

In [92]:
actions = params.keys()

for idx, file in enumerate(os.listdir('data_folder/src/pro/')):
    src_dir = os.path.join('data_folder/src/pro/', file)
    print(src_dir)
    ac = ActionClassifier()
    pose = PoseDetector()
    for action in actions:
        video = cv2.VideoCapture(src_dir)
        while video.isOpened():
            read_ok, frame = video.read()
            if not read_ok:
#                 print('TRK Complete!')
                break
            cnt = video.get(cv2.CAP_PROP_POS_FRAMES)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, dsize=(0, 0), fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
            pose_obj = pose.trk_process(image=frame)
            pose.get_coordis_3d(pose_obj)
            landmarked_img = pose.drawing_with_pose(frame, pose_obj)
            ac.update_information(action=action,
                                parameter=params,
                                pose_coordis=pose.coordinates,
                                num_frame=cnt, image=landmarked_img)
    with open('data_folder/parameter/label_rule_value.json', 'rb') as r:
        label_rule = json.load(r)
    esti_rule = rule.rule_structure
    for action in ac.esti_inform:
        esti_rule = rule.handle(action=action, 
                        value=esti_rule, 
                        coordis=ac.esti_inform[action]['coordinate'])
    buf = './data_folder/labels/pro_jsons/' + str(idx) + '.json'
    with open(buf, 'w',encoding='utf-8') as f:
        json.dump(esti_rule, f, indent=4, ensure_ascii=False)
    del(esti_rule)
    print('-' * 20)

data_folder/src/pro/#shorts #골프스윙 #슬로우모션.mp4
--------------------
data_folder/src/pro/[골프스윙] 7번아이언 슬로우모션.mp4
--------------------
data_folder/src/pro/[우태윤프로] 정면 슬로우모션 스윙 (1).mp4
--------------------
data_folder/src/pro/[전지훈프로]드라이버스윙 정면 슬로우모션 (1).mp4
--------------------
data_folder/src/pro/[전지훈프로]드라이버스윙 정면 슬로우모션.mp4
--------------------
data_folder/src/pro/드라이버 슬로우 정면샷!!💯💯이제 드디어 알것 같아요_👍👍👍 ｜ 예쁜 골프스윙TV #shorts #golf #golfswing.mp4
--------------------
data_folder/src/pro/드라이버 정면슬로우 최고네요!!!👍👍👍👍 ｜ 예쁜 골프스윙TV #shorts #golf #golfswing.mp4
--------------------
data_folder/src/pro/맥길로이 드라이버 슬로모션 🔥🔥.mp4
--------------------
data_folder/src/pro/아이언 정면 슬로우모션.mp4
--------------------
data_folder/src/pro/올바른 하체쓰임 정면 슬로우!!🔥🔥🔥🔥 ｜ 예쁜스윙TV #shorts #golf #golfswing.mp4
--------------------
